<a href="https://colab.research.google.com/github/JVengava/CS6910-DeepLearningAssignments/blob/main/Assignment2/PartA/Assignment2_ObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random

In [ ]:
!pip install wandb
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import keras

# keras pre-trained models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception


from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, BatchNormalization, MaxPooling2D, Activation , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,  Model

import wandb


class ObjectDetection():

    def __init__(self, IMG_SIZE, modelConfigDict, using_pretrained_model = False, base_model = "IRV2" ):
        
        self.no_of_hidden_cnn_layers= modelConfigDict["no_of_hidden_cnn_layers"]
        self.activation = modelConfigDict["activation"]
        self.batch_normalization = modelConfigDict["batch_normalization"]
        self.filter_distribution = modelConfigDict["filter_distribution"]
        self.filter_size = modelConfigDict["filter_size"]
        self.no_of_filters_base  = modelConfigDict["no_of_filters_base"]
        self.dropout_fraction = modelConfigDict["dropout_fraction"]
        self.pool_size = modelConfigDict["pool_size"]
        self.padding = modelConfigDict["padding"]
        self.dense_neurons = modelConfigDict["dense_neurons"]
        self.no_of_classes = modelConfigDict["no_of_classes"]
        self.optimizer = modelConfigDict["optimizer"]
        self.global_average_pooling = modelConfigDict["global_average_pooling"]
        self.batch_normalisation_location = modelConfigDict["batch_normalisation_location"]
        BASE_MODELS = {
                          "IRV2" : IRV2,
                          "IV3" : InceptionV3,
                          "RN50" : ResNet50,
                          "XCPTN" : Xception
                      }      
        
        if using_pretrained_model == True:
            self.base_model = base_model
            if self.base_model == "RN50":
                self.IMG_HEIGHT = 224
                self.IMG_WIDTH = 224
            else:
                self.IMG_HEIGHT = IMG_SIZE[0]
                self.IMG_WIDTH = IMG_SIZE[1]        

        self.IMG_HEIGHT = IMG_SIZE[0]
        self.IMG_WIDTH = IMG_SIZE[1]        
         
        self.input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)


    def build_cnnmodel_conv(self):
        with tf.device('/device:GPU:0'):
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer connecting to input layer
            model.add(Conv2D(self.no_of_filters_base, self.filter_size, padding = self.padding,kernel_initializer = "he_uniform", input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
            model.add(Activation(self.activation))
            
            #batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            #max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            for i in range(self.no_of_hidden_cnn_layers-1):
                #i+2th Convolutional Layer
            
                ## Standard filter distribution - same number of filters in all Convolutional layers
                if self.filter_distribution == "standard":
                    model.add(Conv2D(self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Double filter distribution - double number of filters in each Convolutional layers
                elif self.filter_distribution == "double":
                    model.add(Conv2D(2**(i+1)*self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_distribution == "half":
                    model.add(Conv2D(int(self.no_of_filters_base/2**(i+1)), self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            
                model.add(MaxPooling2D(pool_size=self.pool_size))
                if self.dropout_fraction != None:
                    model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            
            #Final densely connected layers
            if self.global_average_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())

            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            model.add(Dense(self.no_of_classes, activation = 'softmax'))

            return model      
        
    def build_cnnmodel_all(self):
        with tf.device('/device:GPU:0'):
            
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer connecting to input layer
            model.add(Conv2D(self.no_of_filters_base, self.filter_size, padding = self.padding,kernel_initializer = "he_uniform", input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            model.add(Activation(self.activation))
            
            #batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            #max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            for i in range(self.no_of_hidden_cnn_layers-1):
                #i+2th Convolutional Layer
            
                ## Standard filter distribution - same number of filters in all Convolutional layers
                if self.filter_distribution == "standard":
                    model.add(Conv2D(self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                ## Double filter distribution - double number of filters in each Convolutional layers
                elif self.filter_distribution == "double":
                    model.add(Conv2D(2**(i+1)*self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_distribution == "half":
                    model.add(Conv2D(int(self.no_of_filters_base/2**(i+1)), self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            
                model.add(MaxPooling2D(pool_size=self.pool_size))
                if self.dropout_fraction != None:
                    model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            
            #Final densely connected layers
            if self.global_average_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())

            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            model.add(Dense(self.no_of_classes, activation = 'softmax'))
            
            return model      
      
    def build_cnnmodel_dense(self):
        with tf.device('/device:GPU:0'):
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer connecting to input layer
            model.add(Conv2D(self.no_of_filters_base, self.filter_size ,kernel_initializer = "he_uniform",padding = self.padding,input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
            model.add(Activation(self.activation))
            
            #batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
            #max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            for i in range(self.no_of_hidden_cnn_layers-1):
                #i+2th Convolutional Layer
            
                ## Standard filter distribution - same number of filters in all Convolutional layers
                if self.filter_distribution == "standard":
                    model.add(Conv2D(self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Double filter distribution - double number of filters in each Convolutional layers
                elif self.filter_distribution == "double":
                    model.add(Conv2D(2**(i+1)*self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_distribution == "half":
                    model.add(Conv2D(int(self.no_of_filters_base/2**(i+1)),self.filter_size, kernel_initializer = "he_uniform"))
            
                if self.batch_normalisation_location == "Before" and self.batch_normalization: model.add(BatchNormalization())
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_normalization: model.add(BatchNormalization())
                
                model.add(MaxPooling2D(pool_size=self.pool_size))
            
            #Final densely connected layers
            if self.global_average_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())
            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            model.add(Dense(self.no_of_classes, activation = 'softmax'))
          
            return model 


     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 43.5 MB/s 
     |████████████████████████████████| 144 kB 46.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f627b1063d56068b69e15c68226ba765587103df181eed2d6af73caee1ed0523
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
# empty the datasets forlder before downloading the dataset
%cd
%cd .keras/datasets/
!rm -r *

# download data from given url
dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
data_dir = tf.keras.utils.get_file('nature_12K', origin=dataset_url, extract=True)

%cd
%cd .keras/datasets/inaturalist_12K
%mv val test
!mkdir valid

data_folder = '/root/.keras/datasets/inaturalist_12K'
os.chdir(data_folder)

/root
[Errno 2] No such file or directory: '.keras/datasets/'
/root
rm: cannot remove '*': No such file or directory
3816701952/3816687935 [==============================] - 33s 0us/step
/root
/root/.keras/datasets/inaturalist_12K


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib

#wandb logging
import wandb
from wandb.keras import WandbCallback

IMG_SIZE = (128,128)


#sweep config
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': [3],
        's': [2]
  },
  "parameters": {
        
        "activation":{
            "values": ["relu", "elu", "selu"]
        },
        "filter_size": {
            "values": [(2,2), (3,3), (4,4)]
        },
        "batch_size": {
            "values": [32, 64]
        },
        "padding": {
            "values": ["same","valid"]
        },
        "data_augmentation": {
            "values": [True, False]
        },
        "optimizer": {
            "values": ["sgd", "adam", "rmsprop", "nadam"]
        },
        "batch_normalization": {
            "values": [True, False]
        },
        "batch_normalisation_location": {
            "values": ["Before", "After"]
        },
        "no_of_filters_base": {
            "values": [32, 64]
        },
        "dense_neurons": {
            "values": [32, 64, 128]
        },   
        "dropout_location": {
            "values": ["conv","dense","all"]
        },
        "dropout_fraction": {
            "values": [None, 0.2,0.3]
        },  
        "global_average_pooling": {
            "values": [False,True]
        },        
    }
}
sweep_id = wandb.sweep(sweep_config,project='CS6910 _Assignment2', entity='cs21z032_cs22z005')

#train function
def train():

        
    config_defaults = dict(
            no_of_hidden_cnn_layers = 5 ,
            activation = 'relu',
            batch_normalization = True,
            batch_normalisation_location = "After",
            filter_distribution = "double" ,
            filter_size = (3,3),
            no_of_filters_base  = 32,
            dropout_fraction = None,
            dropout_location = "dense",
            pool_size = (2,2),
            padding = 'same',
            dense_neurons = 128,
            no_of_classes = 10,
            optimizer = 'adam',
            epochs = 5,
            batch_size = 32, 
            data_augmentation = False,
            global_average_pooling = True,
            img_size = IMG_SIZE
        ) 

    
    wandb.init( config = config_defaults)
    #wandb.init(project = 'CS6910 _Assignment2', config = config_defaults, entity='cs21z032_cs22z005')        
    CONFIG = wandb.config


    wandb.run.name = "OBJDET_" + str(CONFIG.no_of_hidden_cnn_layers) + "_dn_" + str(CONFIG.dense_neurons) + "_opt_" + CONFIG.optimizer + "_dro_" + str(CONFIG.dropout_fraction) + "_bs_"+str(CONFIG.batch_size) + "_fd_" + CONFIG.filter_distribution + "_bnl_" + CONFIG.batch_normalisation_location + "_dpl_" + CONFIG.dropout_location

    data_augmentation = CONFIG.data_augmentation

    
    BATCH_SIZE = CONFIG.batch_size


    if data_augmentation == True:

    #Faster Alternative
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                validation_split = 0.1,
                shear_range=0.2,
                zoom_range=0.2,
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False
                )
    else:
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split = 0.1)

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    
    train_ds = train_datagen.flow_from_directory(
        data_folder+'/train',
        subset='training',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 123)
        
    validation_ds = train_datagen.flow_from_directory(
            data_folder+'/train',
            target_size=IMG_SIZE,
            subset = 'validation',
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)


            
    test_ds = test_datagen.flow_from_directory(
            data_folder+'/test',
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)



    with tf.device('/device:GPU:0'):        
        objDetn = ObjectDetection(CONFIG.img_size, CONFIG )
        if CONFIG.dropout_location == "all":
            model = objDetn.build_cnnmodel_all()
        elif CONFIG.dropout_location == "conv":
            model = objDetn.build_cnnmodel_conv()
        elif CONFIG.dropout_location == "dense":
            model = objDetn.build_cnnmodel_dense()
        
        model.summary()



        model.compile(
        optimizer=CONFIG.optimizer,  # Optimizer
        # Loss function to minimize
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),#'categorical_crossentropy',
        # List of metrics to monitor
        metrics=['accuracy'],
        )
      
        history = model.fit(
                        train_ds,
                        steps_per_epoch = train_ds.samples // CONFIG.batch_size,
                        validation_data = validation_ds, 
                        validation_steps = validation_ds.samples // CONFIG.batch_size,
                        epochs = CONFIG.epochs, 
                        callbacks=[WandbCallback()]
                        )

        model.save('./TrainedModel/'+wandb.run.name)
        wandb.finish()
        return model, history

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. {'min_iter': [3], 's': [2], 'type': 'hyperband'} is not valid under any of the given schemas


Create sweep with ID: rvwkmlgh
Sweep URL: https://wandb.ai/cs21z032_cs22z005/CS6910%20_Assignment2/sweeps/rvwkmlgh


In [ ]:
wandb.agent(sweep_id, train, count = 100)

wandb: Agent Starting Run: i477suye with config:
wandb: 	activation: elu
wandb: 	batch_normalisation_location: After
wandb: 	batch_normalization: False
wandb: 	batch_size: 32
wandb: 	data_augmentation: True
wandb: 	dense_neurons: 64
wandb: 	dropout_fraction: None
wandb: 	dropout_location: all
wandb: 	filter_size: [4, 4]
wandb: 	global_average_pooling: False
wandb: 	number_of_filters_base: 64
wandb: 	optimizer: nadam
wandb: 	padding: valid
wandb: Currently logged in as: jvengava (use `wandb login --relogin` to force relogin)


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 125, 125, 64)      3136      
                                                                 
 activation (Activation)     (None, 125, 125, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 59, 59, 128)       131200    
                                                                 
 activation_1 (Activation)   (None, 59, 59, 128)       0         
                                                            

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


281/281 [==============================] - 178s 609ms/step - loss: 2.3170 - accuracy: 0.1018 - val_loss: 2.3036 - val_accuracy: 0.0998 - _timestamp: 1648962106.0000 - _runtime: 187.0000
Epoch 2/5
281/281 [==============================] - 171s 607ms/step - loss: 2.3061 - accuracy: 0.0948 - val_loss: 2.3052 - val_accuracy: 0.0998 - _timestamp: 1648962305.0000 - _runtime: 386.0000
Epoch 3/5
281/281 [==============================] - 170s 606ms/step - loss: 2.3068 - accuracy: 0.1002 - val_loss: 2.3042 - val_accuracy: 0.1008 - _timestamp: 1648962506.0000 - _runtime: 587.0000
Epoch 4/5
281/281 [==============================] - 168s 598ms/step - loss: 2.3065 - accuracy: 0.0941 - val_loss: 2.3037 - val_accuracy: 0.0988 - _timestamp: 1648962674.0000 - _runtime: 755.0000
Epoch 5/5
281/281 [==============================] - 167s 595ms/step - loss: 2.3067 - accuracy: 0.0972 - val_loss: 2.3035 - val_accuracy: 0.1008 - _timestamp: 1648962875.0000 - _runtime: 956.0000
INFO:tensorflow:Assets written

accuracy,█▂▇▁▄
epoch,▁▃▅▆█
loss,█▁▁▁▁
val_accuracy,▅▅█▁█
val_loss,▂█▄▂▁
accuracy,0.09723
best_epoch,4
best_val_loss,2.30353
epoch,4
loss,2.30673
val_accuracy,0.10081


...

accuracy,▁▄▅▆█
epoch,▁▃▅▆█
loss,█▅▄▂▁
val_accuracy,▁▄▇▂█
val_loss,█▄▂▇▁
accuracy,0.34837
best_epoch,4
best_val_loss,1.91
epoch,4
loss,1.8621
val_accuracy,0.31458


wandb: Agent Starting Run: 5rjep5z3 with config:
wandb: 	activation: relu
wandb: 	batch_normalisation_location: After
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dense_neurons: 128
wandb: 	dropout_fraction: 0.2
wandb: 	dropout_location: conv
wandb: 	filter_size: [3, 3]
wandb: 	global_average_pooling: True
wandb: 	number_of_filters_base: 64
wandb: 	optimizer: nadam
wandb: 	padding: valid


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 activation (Activation)     (None, 126, 126, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 128)       73856     
                                                            

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


140/140 [==============================] - 160s 1s/step - loss: 2.3295 - accuracy: 0.0938 - val_loss: 2.3087 - val_accuracy: 0.1000 - _timestamp: 1648977926.0000 - _runtime: 170.0000
Epoch 2/5
140/140 [==============================] - 154s 1s/step - loss: 2.3078 - accuracy: 0.1003 - val_loss: 2.3048 - val_accuracy: 0.0990 - _timestamp: 1648978125.0000 - _runtime: 369.0000
Epoch 3/5
106/140 [=====================>........] - ETA: 35s - loss: 2.3093 - accuracy: 0.0985

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn import metrics

import pandas as pd
import os
import cv2


import wandb
from wandb.keras import WandbCallback

#To let the gpu memory utilisiation grow as per requirement
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
#Invalid device or cannot modify virtual devices once initialized.
    pass

#Note: Wandb logging has been removed from the script for easier running
    

# Dataset loading function
def load_dataset_batch(DATAPATH, IMG_HEIGHT, IMG_WIDTH, batch_size = None): 
             
    img_data_array=[] 
    class_name=[] 

    for dir1 in np.sort(os.listdir(DATAPATH)): 
     if dir1[0] != ".": 
         ctr = 0 
         for file in np.sort(os.listdir(os.path.join(DATAPATH, dir1))): 
             ctr += 1 
             if batch_size == None:     
                 image_path= os.path.join(DATAPATH, dir1,  file) 
                 image= cv2.imread( image_path, cv2.COLOR_BGR2RGB) 
                 image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA) 
                 image=np.array(image) 
                 image = image.astype('float32') 
                 image /= 255  
                 img_data_array.append(image) 
                 class_name.append(dir1) 
             else: 
                 if ctr <= batch_size: 
                     image_path= os.path.join(DATAPATH, dir1,  file) 
                     image= cv2.imread( image_path, cv2.COLOR_BGR2RGB) 
                     image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA) 
                     image=np.array(image) 
                     image = image.astype('float32') 
                     image /= 255  
                     img_data_array.append(image) 
                     class_name.append(dir1) 
    return img_data_array, class_name 
          
#class names:
def class_names(DATAPATH):
   
    class_name=[]
    
    for dir1 in np.sort(os.listdir(DATAPATH)):
        class_name.append(dir1)
    return class_name


class_names = class_names(data_folder+"/test/")
target_dict = {k: v for v, k in enumerate(np.unique(class_names))} 
class_label_names_dict = {str(k): v for k, v in enumerate(np.unique(class_names))} 

# The dimensions of our input image
img_width,img_height = 128, 128

img_array, class_labels = load_dataset_batch(data_folder+"/test/", 128,128) 
img_array = np.array([img_array]).reshape(len(img_array), img_width, img_height,3)
class_labels_num = [target_dict[class_labels[i]] for i in range(len(class_labels))] 


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_ds = test_datagen.flow_from_directory(
            data_folder+'/test',
            target_size=(img_width,img_height),
            batch_size=32,
            class_mode='categorical',
            shuffle = False)
# Our target layer: we will visualize the filters from this layer.

#Model trained from scratch
source_model = keras.models.load_model(data_folder+"/TrainedModel/") #Load the best trained model
#Transfer learned model
#source_model = keras.models.load_model("./TrainedModel/Best_TransferlearntModel") #Load the best trained model

# See `model.summary()` for list of layer names, if you want to change this.
source_model.summary()
layer_dict = {layer.name:layer for layer in source_model.layers}
print(layer_dict)

test_ds2 = test_datagen.flow_from_directory(
            data_folder+'/test',
            target_size=(img_width,img_height),
            batch_size=32,
            class_mode='categorical',
            shuffle = True, seed=1234)
            
#Test loss and accuracy on the shuffled test dataset            
history = source_model.evaluate(test_ds2)


#Sample image predictions
ROWS = 3 
COLUMNS = 10  
ix = 1 
for i in range(ROWS): 
    for j in range(COLUMNS): 
        # specify subplot and turn of axis 
        idx = np.random.choice(len(test_ds[4*j][0])) 
        img = test_ds[4*j][0][idx] 
        ax = plt.subplot(ROWS, COLUMNS, ix) 
        ax.set_xticks([]) 
        ax.set_yticks([]) 
        # plot filter channel in grayscale 
        plt.imshow(img) 
        plt.xlabel(
                    "True: " + class_label_names_dict[str(np.argmax(test_ds[4*j][1][idx]))] +"\n" + "Pred: " + 
                    class_label_names_dict[str(np.argmax(source_model.predict(img.reshape(1,128,128,3))))]
                   )     
        ix += 1 
plt.show()

Found 2000 images belonging to 10 classes.


OSError: ignored